<a href="https://colab.research.google.com/github/BossRobin/DeepLearningAlgorithmsByPytorch/blob/master/BiRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms # transforms.ToTensor()把一个取值范围是[0,255]的PIL.Image 转换成 Tensor

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
sequence_length = 28 # 序列长度，即一句话有多少个单词。在这里因为输入的是28*28的图像，所以理解为序列有28个单词，每个单词有28维
input_size = 28 # 输入层节点数，对应输入维度
hidden_size = 128 # 隐藏层节点数
num_layers = 2 # 2层LSTM
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [0]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',train=True,transform=transforms.ToTensor(),download=True) 
# train=True代表这个数据集作为训练集，download=True是要下载下来

test_dataset = torchvision.datasets.MNIST(root='../../data/',train=False,transform=transforms.ToTensor()) 
# train=False代表这个数据集作为测试集

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size, shuffle=True) 
# 上面的Dataset只负责数据的抽象，一次调用getitem只返回一个样本。
# 前面提到过，在训练神经网络时，最好是对一个batch的数据进行操作，同时还需要对数据进行shuffle和并行加速等。对此，PyTorch提供了DataLoader帮助我们实现这些功能。

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=False)

In [0]:
class BiRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(BiRNN, self).__init__() 
    # 使得BiRNN的实例对象可以调用父类__init__()中的属性，即执行了父类的__init__()方法，并将其中的属性附给当前子类的实例对象
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True) 
    # batch_first的意思是在传入数据时，batch_size作为第一维。bidirectional表示是否为双向LSTM
    self.fc = nn.Linear(hidden_size*2, num_classes) 
    # hidden_size*2是因为双向lstm的隐藏层输出是隐藏层节点数的2倍
  def forward(self, x):
    h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) 
    # h_0是格式为(num_layers * num_directions, batch, hidden_size)的tensor 它包含batch中每个元素的最初的隐态，若为双向lstm num_directions=2 否则=1
    c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) 
    # c_0是格式为(seq_len, batch, input_size）的tensor 它包含batch中每个元素最初的cell state，若h_0和c_0不提供，则默认为0

    out, _ = self.lstm(x, (h0, c0)) # lstm的输入格式 
    # out是形状为(seq_len, batch, num_directions*hidden_size）的tensor，包含输出特征h_t(源于LSTM每个t的最后一层)
    # h_n是形状为(num_layers * num_directions, batch, hidden_size)的tensor， 包含t=seq_len（即序列末尾）的隐态值
    # c_n是形状为(num_layers * num_directions, batch, hidden_size)的tensor， 包含t=seq_len（即序列末尾）的cell值
    out = self.fc(out[:, -1, :]) 
    # 去掉out的中间维，即seq_len(因为batch_first=True，所以中间为seq_len。因为序列长度在全连接层用不到，所以去掉)。由size(100, 28, 256)变为size(100, 256)
    return out

In [0]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device) # to(device)是指定将model放到cpu上或者gpu上计算

In [58]:
criterion = nn.CrossEntropyLoss() # 交叉熵损失
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader) # 有多少个batch
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, sequence_length, input_size).to(device) 
    # 因为从train_loader中得到的images的size为(100, 1, 28, 28)，所以要把1去掉
    labels = labels.to(device)
    outputs = model(images) 
    # model(images)相当于model.__call__(images),而在__call__中，调用了forward的方法。所以model(images)就相当于进行了前向传播。
    loss = criterion(outputs, labels) 
    # outputs的维度是torch.Size([100, 10])labels的维度是torch.Size([100])。因为CrossEntropyLoss()方法进行了综合处理，所以直接这么输入就可以得到loss，可视为无缝连接。
    optimizer.zero_grad() # 每次反向传播都需要将梯度归零，否则会累加，因为是一个批次一个批次进行梯度下降的，应该互不影响。
    loss.backward() # 反向传播，backward函数自动实现的
    optimizer.step() # 梯度更新

    if (i+1) % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item())) #.item()意思是将tensor值转化为python值


torch.Size([100, 256])
torch.Size([100, 10]) torch.Size([100])
torch.Size([100, 256])
torch.Size([100, 10]) torch.Size([100])


In [0]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, sequence_length, input_size).to(device)

    labels = labels.to(device)
    outputs = model(images)
    print(outputs.size())
    break
    _, predicted = torch.max(outputs.data, 1) # outputs.data的维度是（100，10），torch.max(a, b)的意思是按照a的第b维求最大值。得到的结果是输出的labels中的最大概率以及对应的label

    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy of the model on the 10000 test images: {}'.format(100 * correct / total))